# BerTopic
based on 1st starting posts in "Hotline" Russian women forum eva.ru

In [ ]:
!pip install bertopic

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from bertopic import BERTopic

In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
Data = '/content/drive/MyDrive/Colab Notebooks/Projects/eva/Data/'

Messages_filename='63.csv'
Messages_full_filename=os.path.join(Data, Messages_filename)

Messages_topics_filename='63Topics.csv'
Messages_topics_full_filename=os.path.join(Data, Messages_topics_filename)

text='message'
preprocessed_text='preprocessed_message'

Models = '/content/drive/MyDrive/Colab Notebooks/Projects/eva/Models/ChapterTopics/'
ModelName='63'


In [ ]:
df=pd.read_csv(Messages_full_filename, error_bad_lines=False, index_col=False)

In [ ]:
#change text column type to string
df[text]=df[text].values.astype(str)

In [ ]:
import nltk

In [ ]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
ru_stopwords = stopwords.words('russian')
eng_stopwords = stopwords.words('english')
add_stopwords = ['ай',
'ну',
'по',
'это',
'так',
'что',
'вот',
'кого',
'то',
'кто',
'все',
'мочь',
'всё',
'ваш',
'спасибо',
'пожалуйста',
'посоветуйте',
'вопрос',
'весь',
'ав',
'очень',
'ещё',
'просто',
'свой',
'собственно',
'сабж',
'согласный',
'ть',
'подсказать',
'раздел',
'тема',
'ред',
'nan'
	]
stopwords = eng_stopwords + ru_stopwords + add_stopwords

from collections import Counter
stopwords_dict = Counter(stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 3.4 MB/s eta 0:00:011
     |████████████████████████████████| 8.2 MB 14.0 MB/s 


In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return  ' '.join(res)
def text_preprocess(row):
  s=lemmatize(row[text])
  return ' '.join([word for word in tokenizer.tokenize(s) if word not in stopwords_dict])

In [ ]:
df[preprocessed_text]=df.apply(text_preprocess,axis=1)

In [ ]:
df[preprocessed_text]=df[preprocessed_text].values.astype(str)

In [ ]:
def num_words(row):
  words=row[preprocessed_text].split()
  return len(words)

In [ ]:
df['num_words']=df.apply(num_words,axis=1)

In [ ]:
df[[preprocessed_text, 'num_words']].sort_values('num_words', ascending=True).head(10)

,preprocessed_message,num_words
13868,ситуация сегодня умереть наш любимый бабушка б...,26
35528,96 летний знаменитый актёр владимир этуш умере...,26
1423,купить дочий ева костюм зимний врод понравился...,26
41130,работаю давно попросить деньга важный вещь нач...,26
23912,муж работать госструктура снова четверг добров...,26
22884,спросить большой народ обитает скорее психолог...,26
7297,возможность избавиться общение любить нынешний...,26
40460,подруга родить несколько месяц назад пора врем...,26
37415,девы скажите мнению мужик потратить разговор ж...,26
26950,развеять миф притяжение противоположностейпсих...,26


In [ ]:
df[[preprocessed_text, 'num_words']].sort_values('num_words', ascending=True).tail(10)

,preprocessed_message,num_words
12964,переносить пока работу решить работать удаленн...,49
13049,стадия пересмотр отношение родными суть решено...,49
29840,воплощение баловень судьба яв ся олег табаков ...,49
13070,прилететь сегодня домой стоить ждать чемодан р...,49
13158,читать несколько подряд детский др удвиляться ...,49
29751,ребёнок активно увлечь одобряете например двое...,49
39659,накручивать вчерашний взводе расстроиться сын ...,49
13237,слышать разговор телефонный знакомый дочкой ст...,49
30380,немного тупик ребёнок подруга неожиданно обнар...,49
21649,знаю знать задолбать полеты чес словый последн...,49


In [ ]:
df.to_csv(Messages_full_filename, header=True, index=False)

In [ ]:
df=df[((df['num_words']>25) & (df['num_words']<50))]

In [ ]:
len(df)

9802

In [ ]:
data = df[preprocessed_text].tolist()
#165
#model = BERTopic(embedding_model='blinoff/roberta-base-russian-v0')
#203 , nr_topics=50 language="Russian", calculate_probabilities=True, nr_topics=7
model = BERTopic(language="Russian", nr_topics=10) 
topics, probabilities = model.fit_transform(data)
topics_df=model.get_topic_info()
topics_df.head(10)

,Topic,Count,Name
0,0,9715,0_ребёнок_муж_год_хотеть
1,1,44,1_собака_собаку_собачка_гулять
2,2,31,2_кошка_кот_муж_котик
3,-1,12,-1_котик_собака_собак_дверь


In [ ]:
Model_full_filename=os.path.join(Models, ModelName)
model.save(Model_full_filename)

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.

